# Just for fun - 00

* Concept: Try to make two LLMs discuss together with simple `ConversationBufferMemory`.
* LLMs are named:
  1. `Alice`
  2. `Bob`

## Initial settings

Set APIs keys (LLM and LangSmith) and retrieve model's values.

In [1]:
from custom_features.models import MODELS
from custom_features.params_setting_fct import set_api_keys, set_langsmith

set_api_keys()
set_langsmith(langsmith_project_name="just_for_fun_00")

In [2]:
model = MODELS[0]["model"]
model_provider = MODELS[0]["model_provider"]

## Rate limite

MistralAI allows only 1 request per second.

In [3]:
from langchain_core.rate_limiters import InMemoryRateLimiter

In [32]:
rate_limiter = InMemoryRateLimiter(requests_per_second=0.2,
                                   check_every_n_seconds=0.1,
                                   max_bucket_size=1)

## Players

In [33]:
# from langchain.chat_models import init_chat_model
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage, HumanMessage

### Alice

Role: Alice wants to learn how to grow mushrooms

In [ ]:
# alice_llm = init_chat_model(model=model, model_provider=model_provider)
alice_llm = ChatMistralAI(model=model, max_retries=5, rate_limiter=rate_limiter)

In [35]:
alice_system_msg = SystemMessage("""You are Alice.
                                 Your goal is to grow mushrooms but you do not know how to do that.
                                 You only speak French.
                                 You stop asking about mushrooms growing when you think you have enough informations and stop the conversation.
                                 """)

In [10]:
alice_response = alice_llm.invoke([alice_system_msg]).content
alice_response

"Bonjour! Je suis Alice et je veux apprendre à cultiver des champignons. Je n'ai pas de connaissances sur le sujet et je parle français. Pourrais-tu me donner quelques informations sur les conditions idéales pour cultiver des champignons et des instructions simples pour commencer ? Merci !"

### Bob

Role: Bob answers any Alice's question

In [36]:
bob_llm = ChatMistralAI(model=model, max_retries=5, rate_limiter=rate_limiter)

In [37]:
bob_system_msg = SystemMessage("""Your name is Bob.
                               Stop the conversation when the user tells you that is good for him."""
                               )

In [14]:
bob_response = bob_llm.invoke([bob_system_msg, alice_response])
bob_response

AIMessage(content="Bonjour Alice! Je suis Bob et je suis heureux de vous aider à apprendre à cultiver des champignons!\n\nPour commencer, il est important de connaître les conditions idéales pour cultiver des champignons. Voici quelques informations importantes:\n\n1. Le substrat: Les champignons ont besoin d'un substrat pour croître, il peut être constitué de terre, de compost, de bois, de foin ou d'autres matériaux organiques. Il est important de choisir un substrat qui est riche en nutriments et qui offre une bonne structure pour le champignon à pousser.\n2. La température: Les champignons ont des préférences en matière de température. Par exemple, les champignons de type Agaricus (comme les champignons de Paris) croissent mieux entre 20 et 25 degrés Celsius, tandis que les champignons de type Psilocybe préfèrent des températures plus froides, entre 10 et 15 degrés Celsius. Il est donc important de choisir un climat approprié pour le type de champignon que vous souhaitez cultiver.\n

In [16]:
alice_llm.invoke(bob_response.content)

AIMessage(content="Bonjour Bob! Merci d'aider à apprendre à cultiver des champignons. Je suis en train de suivre vos instructions et je suis prêt à commencer à cultiver mes propres champignons.\n\nJe suis impressionné par la variété de conditions idéales nécessaires pour cultiver des champignons. Je comprends maintenant pourquoi il est important de choisir un substrat approprié, de réguler l'humidité, de maintenir une température appropriée et de fournir un environnement lumineux ou sombre en fonction du type de champignon que l'on souhaite cultiver.\n\nJe suis prêt à choisir un substrat approprié pour mes champignons préférés, à étaler le substrat sur une surface propre et lisse, à inoculer le substrat avec du mycélium, à couvrir le substrat avec une couche d'éponge ou de tissu pour maintenir l'humidité, à mettre le substrat dans un environnement approprié et à attendre que le champignon pousse et croisse.\n\nJe suis également prêt à harverster le champignon lorsqu'il est à maturité, 

In [39]:
alice_response = alice_llm.invoke([alice_system_msg]).content
print(f"<<< Alice 0 >>>\n{alice_response}\n")

for _ in range(10):
    # if _ == 0:
    bob_response = bob_llm.invoke([bob_system_msg, alice_response]).content
    # else:
    #     bob_response = bob_llm.invoke([alice_response]).content
    print(f"<<< Bob {_} >>>\n{bob_response}\n")
    alice_response = alice_llm.invoke([alice_system_msg, bob_response]).content
    print(f"<<< Alice {_ +1} >>>\n{alice_response}\n")

<<< Alice 0 >>>
Bonjour ! Je suis Alice et je veux croître des champignons, mais je ne sais pas comment le faire. Peut-être pouvez-vous me donner certaines informations sur les conditions nécessaires pour croître des champignons ?

Je connais que les champignons ont besoin d'un milieu humide et chaud pour croître. Dois-je trouver un endroit naturel ou peut-être peut-être je peux créer mon propre milieu pour les cultiver ?

Je suis prête à apprendre tout ce que vous pouvez me dire sur les champignons et les conditions de culture.

<<< Bob 0 >>>
Bonjour Alice, je suis Bob. Je suis heureux de vous aider à cultiver des champignons.

Pour commencer, il est important de savoir que les champignons ont besoin d'un milieu humide, frais et pauvre en nutriments pour croître. Les champignons peuvent être trouvés dans divers types de milieux naturels tels que les forêts, les bois morts, les décompositions organiques et les zones humides.

Si vous souhaitez cultiver des champignons à la maison, vous

## Automated conversation between Alice and Bob about mushrooms with `ConversationBufferMemory`

In [25]:
from langchain_core.prompts import ChatPromptTemplate

In [17]:
memory = ConversationBufferMemory()

C:\Users\nanar\AppData\Local\Temp\ipykernel_24920\995385594.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [26]:
alice_prompt_template = ChatPromptTemplate.from_messages(
    [alice_system_msg]
)

In [27]:
alice_llm = ChatMistralAI(model_name=model, max_retries=5, rate_limiter=rate_limiter, temperature=0.7)
alice_conversation = ConversationChain(llm=alice_llm,
                                       memory=memory,
                                       verbose=True,
                                       )

In [21]:
bob_llm = ChatMistralAI(model_name=model, max_retries=5, rate_limiter=rate_limiter, temperature=0.1)
bob_conversation = ConversationChain(llm=bob_llm,
                                     memory=memory,
                                     verbose=True,
                                     )

In [28]:
alice_response = alice_conversation.invoke(alice_prompt_template).content
print(f"<<< ALICE >>>:\n{alice_response}")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: input_variables=[] input_types={} partial_variables={} messages=[SystemMessage(content='You are Alice.\n                                 Your goal is to grow mushrooms but you do not know how to do that.\n                                 You only speak French.\n                                 You stop asking about mushrooms growing when you think you have enough informations and stop the conversation.\n                                 ', additional_kwargs={}, response_metadata={})]
AI:


ValidationError: 21 validation errors for HumanMessage
content.str
  Input should be a valid string [type=string_type, input_value=ChatPromptTemplate(input_... response_metadata={})]), input_type=ChatPromptTemplate]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
content.list[union[str,dict[any,any]]].0.str
  Input should be a valid string [type=string_type, input_value=('name', None), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
content.list[union[str,dict[any,any]]].0.dict[any,any]
  Input should be a valid dictionary [type=dict_type, input_value=('name', None), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.10/v/dict_type
content.list[union[str,dict[any,any]]].1.str
  Input should be a valid string [type=string_type, input_value=('input_variables', []), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
content.list